In [ ]:
import cv2
from keras.models import model_from_json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Load the model
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

# Load Haar Cascade for face detection
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

# Function to preprocess images
def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

# Start webcam
webcam = cv2.VideoCapture(0)

# Labels for emotions
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

print("Press 'Stop' or interrupt the kernel to exit.")
try:
    while True:
        # Read frame from webcam
        ret, im = webcam.read()
        if not ret:
            print("Failed to capture image from webcam.")
            break

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Process each face
        for (p, q, r, s) in faces:
            # Extract the face region
            face_image = gray[q:q+s, p:p+r]
            face_image = cv2.resize(face_image, (48, 48))

            # Prepare the image for prediction
            img = extract_features(face_image)
            pred = model.predict(img)
            prediction_label = labels[pred.argmax()]

            # Draw rectangle and label on the original frame
            cv2.rectangle(im, (p, q), (p + r, q + s), (255, 0, 0), 2)
            cv2.putText(im, prediction_label, (p, q - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame in the notebook
        clear_output(wait=True)
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        display(plt.gcf())

except KeyboardInterrupt: 
    print("Exiting...")

# Release the webcam
webcam.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.saving import register_keras_serializable

# Register Sequential as a serializable class
register_keras_serializable()(Sequential)

# Dictionary mapping emotion labels to emotions
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# Load the model architecture from JSON
with open(r'C:/Users/wanai/EmotionDetectionFacialExpression/prj1/Face_Emotion_Recognition_Machine_Learning-main/emotion_model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

# Pass `custom_objects` to recognize custom classes during deserialization
emotion_model = model_from_json(loaded_model_json, custom_objects={"Sequential": Sequential})

# Load the model weights from HDF5 file
weights_path = r'C:/Users/wanai/EmotionDetectionFacialExpression/prj1/Face_Emotion_Recognition_Machine_Learning-main/emotion_model.h5'
emotion_model.load_weights(weights_path)
print("Loaded model from disk")

# Start the webcam feed or use a video file path
cap = cv2.VideoCapture(r"C:/Users/wanai/EmotionDetectionFacialExpression/prj1/emotion-sample.mp4")

# Haar cascade for face detection
face_detector = cv2.CascadeClassifier(r'C:/Users/wanai/EmotionDetectionFacialExpression/prj1/Face_Emotion_Recognition_Machine_Learning-main/haarcascade_frontalface_default.xml')

while True:
    # Read each frame from the video
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (1280, 720))

    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in num_faces:
        # Draw a rectangle around each detected face
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)

        # Preprocess the face region for emotion prediction
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # Predict the emotion
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    
    # Display the video with annotations
    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Loaded model from disk
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 


KeyboardInterrupt

